## Pretrained InceptionResNetV2 model

In [0]:
#InceptionResNetV2
#learning rate=0.0002
#learning rate=0.0001 in the following cell will give a better result
%reset -f
import torch
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
def get_data(images):
    train_images = images
    temp_images = []
    
    for i in range(len(train_images)):
        img_idx = i
        temp_images.append([])
        temp_images[i] = cv2.resize(train_images[img_idx],(75,75))
        temp_images[i] = cv2.cvtColor(temp_images[i],cv2.COLOR_GRAY2BGR)
        
        image = np.ascontiguousarray(train_images[img_idx], dtype=np.uint8)

        ret, threshed_img = cv2.threshold(image,
                        240, 255, cv2.THRESH_BINARY)
        img, contours, her = cv2.findContours(threshed_img, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        
        largest_dim = {"x":0, "y":0, "w":0, "h":0}
        largest_area = 0
        for num in contours:
            x,y,w,h = cv2.boundingRect(num)
            
            temp = 0
            
            if w > h:
                temp=w
            else:
                temp=h
                
            if temp*temp > largest_area:
                largest_area = temp*temp
                largest_dim["x"] = x
                largest_dim["y"] = y
                largest_dim["w"] = w
                largest_dim["h"] = h
        

        b = threshed_img[largest_dim["y"]:largest_dim["y"]+largest_dim["h"],largest_dim["x"]:largest_dim["x"]+largest_dim["w"]]
    
    temp_images = np.asarray(temp_images, dtype=np.float32)
    
    
    return temp_images
  
def split_data(train_size, valid_size, test_size):
    images = pd.read_pickle('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_images.pkl')
    labels = pd.read_csv('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_labels.csv')
    return (images[:train_size], images[train_size:train_size+valid_size],images[train_size+valid_size:]),(labels[:train_size], labels[train_size:train_size+valid_size],labels[train_size+valid_size:])


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam, SGD
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(25)


(X_train, X_test, final_images), (train_labels_new, test_labels_new, final_labels)=split_data(35000, 4000, 1000)

X_train = get_data(X_train)
X_test = get_data(X_test)
final_images = get_data(final_images)

y_train = np.array(train_labels_new["Category"].tolist(), dtype="uint8")

y_test = np.array(test_labels_new["Category"].tolist(), dtype="uint8")

final_labels = np.array(final_labels["Category"].tolist(), dtype="uint8")

print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)
print("X_test original shape", X_test.shape)
print("y_test original shape", y_test.shape)

# input image dimensions
img_rows, img_cols = 75, 75

X_train = X_train.reshape(X_train.shape[0], 75, 75, 3)
X_test = X_test.reshape(X_test.shape[0], 75, 75, 3)

final_images = final_images.reshape(final_images.shape[0], 75, 75, 3)


X_train/=255
X_test/=255
final_images/=255

X_train.shape

number_of_classes = 10


Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

# Build InceptionResNetV2 model

from keras.applications.vgg16 import VGG16
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.densenet import DenseNet201
from keras.applications.inception_v3 import InceptionV3
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.xception import Xception



model = InceptionResNetV2(include_top=True, weights=None, input_tensor=None, input_shape=(75,75,3), pooling="avg", classes=10)



model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0002), metrics=['accuracy'])
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08, featurewise_std_normalization=False)

gen.fit(X_train)

train_generator = gen.flow(X_train, Y_train, batch_size=64)
test_generator = gen.flow(X_test, Y_test, batch_size=64)


history = model.fit_generator(train_generator, steps_per_epoch=35000//64, epochs=30,validation_data=test_generator, validation_steps=4000//64)


score = model.evaluate(gen.standardize(X_test), Y_test)
print()
print('Test accuracy: ', score[1])

import pandas as pd
import cv2
import numpy as np
import csv
from google.colab import drive
drive.mount('/content/gdrive')
def write_output_csv(output_file,y_pred):
    with open(output_file, 'w') as csvfile:
        fieldnames = ['ImageId', 'Label']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for index in range(len(y_pred)):
            writer.writerow({'ImageId': index, 'Label': y_pred[index]})

    
    
    return temp_images
test_images = pd.read_pickle('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/test_images.pkl')
X_test = get_data(test_images)
X_test = X_test.reshape(X_test.shape[0], 75, 75, 3)


import numpy as np
final_y = model.predict(X_test)

final_y_result = [np.argmax(row) for row in final_y]

index_list = [p for p in range(len(final_y_result))]

df = pd.DataFrame({'Id':index_list, "Category":final_y_result}, columns=['Id', "Category"])

df.to_csv("final_result.csv",index=False)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Using TensorFlow backend.


X_train original shape (35000, 75, 75, 3)
y_train original shape (35000,)
X_test original shape (4000, 75, 75, 3)
y_test original shape (4000,)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/30
546/546 [==============================] - 396s 726ms/step - loss: 1.4583 - acc: 0.5156 - val_loss: 1.5627 - val_acc: 0.5731
Epoch 2/30
546/546 [==============================] - 350s 641ms/step - loss: 0.5917 - acc: 0.8111 - val_loss: 0.8462 - val_acc: 0.7586
Epoch 3/30
546/546 [==============================] - 348s 637ms/step - loss: 0.3944 - acc: 0.8730 - val_loss: 0.4955 - val_acc: 0.8389
Epoch 4/30
546/546 [==============================] - 348s 637ms/step - loss: 0.2996 - acc: 0.9026 - val_loss: 0.9812 - val_acc: 0.7144
Epoch 5/30
546/546 [==============================] - 348s 637ms/step - loss: 0.2599 - acc: 0.9164 - val_loss: 0.3917 - val_acc: 0.8755
Epoch 6/30
546/546 [==============================] - 34

In [0]:
#learing rate=0.0001
%reset -f
import torch
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
def get_data(images):
    train_images = images
    temp_images = []
    
    for i in range(len(train_images)):
        img_idx = i
        temp_images.append([])
        temp_images[i] = cv2.resize(train_images[img_idx],(75,75))
        temp_images[i] = cv2.cvtColor(temp_images[i],cv2.COLOR_GRAY2BGR)
        
        image = np.ascontiguousarray(train_images[img_idx], dtype=np.uint8)

        ret, threshed_img = cv2.threshold(image,
                        240, 255, cv2.THRESH_BINARY)
        img, contours, her = cv2.findContours(threshed_img, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        
        largest_dim = {"x":0, "y":0, "w":0, "h":0}
        largest_area = 0
        for num in contours:
            x,y,w,h = cv2.boundingRect(num)
            
            temp = 0
            
            if w > h:
                temp=w
            else:
                temp=h
                
            if temp*temp > largest_area:
                largest_area = temp*temp
                largest_dim["x"] = x
                largest_dim["y"] = y
                largest_dim["w"] = w
                largest_dim["h"] = h
        

        b = threshed_img[largest_dim["y"]:largest_dim["y"]+largest_dim["h"],largest_dim["x"]:largest_dim["x"]+largest_dim["w"]]
    
    temp_images = np.asarray(temp_images, dtype=np.float32)
    
    
    return temp_images
  
def split_data(train_size, valid_size, test_size):
    images = pd.read_pickle('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_images.pkl')
    labels = pd.read_csv('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_labels.csv')
    return (images[:train_size], images[train_size:train_size+valid_size],images[train_size+valid_size:]),(labels[:train_size], labels[train_size:train_size+valid_size],labels[train_size+valid_size:])


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam, SGD
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(25)



(X_train, X_test, final_images), (train_labels_new, test_labels_new, final_labels)=split_data(35000, 4000, 1000)

X_train = get_data(X_train)
X_test = get_data(X_test)
final_images = get_data(final_images)

y_train = np.array(train_labels_new["Category"].tolist(), dtype="uint8")

y_test = np.array(test_labels_new["Category"].tolist(), dtype="uint8")

final_labels = np.array(final_labels["Category"].tolist(), dtype="uint8")

print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)
print("X_test original shape", X_test.shape)
print("y_test original shape", y_test.shape)

# input image dimensions
img_rows, img_cols = 75, 75

# the data, split between train and test sets
X_train = X_train.reshape(X_train.shape[0], 75, 75, 3)
X_test = X_test.reshape(X_test.shape[0], 75, 75, 3)

final_images = final_images.reshape(final_images.shape[0], 75, 75, 3)

X_train/=255
X_test/=255
final_images/=255

X_train.shape

number_of_classes = 10

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

# InceptionResNetV2 model
from keras.applications.vgg16 import VGG16
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.densenet import DenseNet201
from keras.applications.inception_v3 import InceptionV3
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.xception import Xception



model = InceptionResNetV2(include_top=True, weights=None, input_tensor=None, input_shape=(75,75,3), pooling="avg", classes=10)



model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08, featurewise_std_normalization=False)

gen.fit(X_train)

train_generator = gen.flow(X_train, Y_train, batch_size=64)
test_generator = gen.flow(X_test, Y_test, batch_size=64)


history = model.fit_generator(train_generator, steps_per_epoch=35000//64, epochs=1,validation_data=test_generator, validation_steps=4000//64)


score = model.evaluate(gen.standardize(X_test), Y_test)
print()
print('Test accuracy: ', score[1])


import pandas as pd
import cv2
import numpy as np
import csv
from google.colab import drive
drive.mount('/content/gdrive')
def write_output_csv(output_file,y_pred):
    with open(output_file, 'w') as csvfile:
        fieldnames = ['ImageId', 'Label']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for index in range(len(y_pred)):
            writer.writerow({'ImageId': index, 'Label': y_pred[index]})
def get_data(images):
    train_images = images
    temp_images = []
    
    for i in range(len(train_images)):
        img_idx = i
        temp_images.append([])
        temp_images[i] = cv2.resize(train_images[img_idx],(75,75))
        temp_images[i] = cv2.cvtColor(temp_images[i],cv2.COLOR_GRAY2BGR)
        
        image = np.ascontiguousarray(train_images[img_idx], dtype=np.uint8)

        ret, threshed_img = cv2.threshold(image,
                        240, 255, cv2.THRESH_BINARY)
        img, contours, her = cv2.findContours(threshed_img, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    
    temp_images = np.asarray(temp_images, dtype=np.float32)
    
    
    return temp_images
test_images = pd.read_pickle('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/test_images.pkl')
X_test = get_data(test_images)
X_test = X_test.reshape(X_test.shape[0], 75, 75, 3)


import numpy as np
final_y = model.predict(X_test)
final_y_result = [np.argmax(row) for row in final_y]
index_list = [p for p in range(len(final_y_result))]
df = pd.DataFrame({'Id':index_list, "Category":final_y_result}, columns=['Id', "Category"])
df.to_csv("final_result.csv",index=False)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Using TensorFlow backend.


X_train original shape (35000, 75, 75, 3)
y_train original shape (35000,)
X_test original shape (4000, 75, 75, 3)
y_test original shape (4000,)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
4000/4000 [==============================] - 14s 4ms/step

Test accuracy:  0.2805
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
